In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
# import loggingfile

# import key - mouse
data = pd.read_csv("./key_mouse.log", encoding = "ISO-8859-1", sep = ",",index_col="time")
data.index = pd.to_datetime(data.index)
# define testdata
### columnname
col_time = "timestamp"
col_perif = "perif"
col_pos = "location"
col_event = "event"

print(data.loc[data["perif"] == "keyboard"])

FileNotFoundError: [Errno 2] No such file or directory: './key_mouse.log'

In [41]:
# define testdata
### columnname
col_time = "timestamp"
col_perif = "perif"
col_loc = "location"
col_event = "event"

data_keyboard = pd.DataFrame(
    data={
        col_time: pd.date_range("2018-01-01", periods=10, freq="H").tolist(),
        col_perif: ["keyboard","keyboard","keyboard","keyboard","keyboard","keyboard","keyboard","keyboard","keyboard","keyboard"],
        col_loc:["e","f","e","e","e","e","f","e","e","e"],
        col_event:["pressed","pressed", "released","pressed","pressed", "released", "released","pressed","pressed", "released",],
    }
)

data_mouse = pd.DataFrame(
    data={
        col_time: pd.date_range("2018-01-01 00:00:01", periods=10, freq="H").tolist(),
        col_perif: ["mouse"]*10,
        col_loc:["pos","pos","pos","pos","pos","pos","lmb","lmb", "lmb","lmb",],
        col_event:[ (0,0),(1,0),(3,1),(1,2),(0,0),(1,2),"pressed", "released","pressed", "released",],
    }
)

data_total = pd.concat([data_keyboard,data_mouse]).set_index(col_time).sort_index()
print(data_total)

                        perif location     event
timestamp                                       
2018-01-01 00:00:00  keyboard        e   pressed
2018-01-01 00:00:01     mouse      pos    (0, 0)
2018-01-01 01:00:00  keyboard        f   pressed
2018-01-01 01:00:01     mouse      pos    (1, 0)
2018-01-01 02:00:00  keyboard        e  released
2018-01-01 02:00:01     mouse      pos    (3, 1)
2018-01-01 03:00:00  keyboard        e   pressed
2018-01-01 03:00:01     mouse      pos    (1, 2)
2018-01-01 04:00:00  keyboard        e   pressed
2018-01-01 04:00:01     mouse      pos    (0, 0)
2018-01-01 05:00:00  keyboard        e  released
2018-01-01 05:00:01     mouse      pos    (1, 2)
2018-01-01 06:00:00  keyboard        f  released
2018-01-01 06:00:01     mouse      lmb   pressed
2018-01-01 07:00:00  keyboard        e   pressed
2018-01-01 07:00:01     mouse      lmb  released
2018-01-01 08:00:00  keyboard        e   pressed
2018-01-01 08:00:01     mouse      lmb   pressed
2018-01-01 09:00:00 

In [50]:
# define Dataobj

class DataPerif:
    # input has to be sorted
    def __init__(self, all_data, tf_sec) -> None:
        # split data
        self.keyboard_data = all_data.loc[all_data[col_perif] == "keyboard"]
        self.analog_data = all_data.loc[all_data[col_perif] == "analog"]
        self.mouse_data = all_data.loc[all_data[col_perif] == "mouse"]
        self.eye_data = all_data.loc[all_data[col_perif] == "eye"]
        
        # get generell info
        self.time_frame = tf_sec
        self.max_time = all_data.index.max()

        # calculate info
        # keyboard
        self.nr_key_strokes = len(self.keyboard_data.loc[self.keyboard_data[col_event] == "pressed"])
        self.key_presses = self.get_full_press()
        self.key_press_time  = pd.Series([tup[1] for tup in self.key_presses]).mean()
        self.key_dead_times = self.get_dead_times()
        self.key_dead_time_avg = pd.Series([tup[0] for tup in self.key_dead_times]).mean()
        self.key_backspaces = 33 # TODO: Is this best way to measure mistakes

        # mouse
        self.avg_distance, self.avg_speed = self.get_move_px()

    def get_full_press(self):
        full_presses = []
        keys_grouped = list(self.keyboard_data.groupby(col_loc))
        
        for key_list in keys_grouped:
            key = key_list[0]
            key_data = key_list[1]
            if len(key_data)>1:

                print(key)
                print(key_data)
                # remove incomplete presses 
                key_data_full_presses = key_data.loc[(key_data[col_event] == "released").idxmin() :(key_data[col_event] == "released")[::-1].idxmax() ]
                
                # remove double press event  TODO maybe add number 
                key_data_rem_duplicate = list(key_data_full_presses.loc[key_data_full_presses[col_event].shift() != key_data_full_presses[col_event]].groupby(col_event))
                
                full_presses += [(key, 
                                key_data_rem_duplicate[1][1].index[num] - key_data_rem_duplicate[0][1].index[num], 
                                key_data_rem_duplicate[0][1].index[num], 
                                key_data_rem_duplicate[1][1].index[num] ) for num, _ in enumerate(key_data_rem_duplicate[1][1].values)]
        return full_presses

    def get_avg_time(self):
        # not used, because its one line
        times = pd.Series([tup[1] for tup in self.key_presses]).mean()
        return times
    
    def get_dead_times(self):
        # TODO define edgecases
        # return max size if no complete presses
        # get complete timeframe
        out_col = "off_time"
        all_times = self.keyboard_data
        all_times = pd.concat([all_times.iloc[[0]], all_times, all_times.iloc[[-1]]])
        all_times[out_col] = True

        # filter the keypresses
        for on_time in self.key_presses:
            all_times.loc[(all_times.index > on_time[2]) & (all_times.index < on_time[3]), out_col] = False
        #print(all_times)

        press_edges = all_times.loc[all_times[out_col].shift() != all_times[out_col]]
        press_edges = press_edges.loc[(press_edges[out_col] == False).idxmin():(press_edges[out_col] == False)[::-1].idxmax() ] # TODO: change when edgecases defined
        print(press_edges)
        
        dead_times=list((press_edges.iloc[2*i+1].name-press_edges.iloc[2*i].name, press_edges.iloc[2*i].name,press_edges.iloc[2*i+1].name,)  for i in range(int(len(press_edges)/2)))
        #print(all_times.loc[out_col == True])
        return dead_times

    def get_move_px(self):
        # get only mouse movemnts
        mouse_move = self.mouse_data.loc[self.mouse_data["location"] == "pos"]
        # remove douplicates in timestamp --> sometimes duplicates occur --> first value is taken
        # fastest method as shown here https://stackoverflow.com/questions/13035764/remove-pandas-rows-with-duplicate-indices
        mouse_remove_doup = mouse_move[~mouse_move.index.duplicated(keep = "first")]
        # calc distance between rows via pythagoras 
        # TODO: split mouse movemnt in swipes (some deadtime between moevments)
        for n,col in enumerate(["x","y"]):
            mouse_remove_doup[col] = mouse_remove_doup['event'].apply(lambda location: location[n])
        mouse_remove_doup["xdiffsq"] = mouse_remove_doup["x"].diff().pow(2)
        mouse_remove_doup["ydiffsq"] = mouse_remove_doup["y"].diff().pow(2)
        mouse_remove_doup["timediff"] = mouse_remove_doup.index.to_series().diff().div(np.timedelta64(1, 's'))
        mouse_remove_doup["distance"] = np.sqrt(mouse_remove_doup["xdiffsq"]+mouse_remove_doup["ydiffsq"])
        mouse_remove_doup["velocity"] = mouse_remove_doup["distance"].div(mouse_remove_doup["timediff"]) 
        #mouse_remove_doup
        
        output = mouse_remove_doup 
        return output["distance"].mean(),output["velocity"].mean() 
        

    def output_string(self):
        # returns console output to test
        print(f'TIME:')
        print(f'Timeframe: {self.time_frame}\nEndtime: {self.max_time}\n')
        print(f'Keyboard:\nN.o. Activations: {self.nr_key_strokes}\nAvg. Presstime: {self.key_press_time}\nAvg. Deadtime: {self.key_dead_time_avg}')
        print(f'Mouse:\nAvg. Speed: {self.avg_speed} px/sec')

class KeyPress:
    # one Keypress
    def __init__(self) -> None:
        pass
class KeyPressLetter:
    # all completed presses of one letter in timeframe
    def __init__(self) -> None:
        self.start_time = True
    


#dat_obj = DataPerif(data,30)
dat_obj = DataPerif(data_total,30)
#print(dat_obj.max_time)

e
                        perif location     event
timestamp                                       
2018-01-01 00:00:00  keyboard        e   pressed
2018-01-01 02:00:00  keyboard        e  released
2018-01-01 03:00:00  keyboard        e   pressed
2018-01-01 04:00:00  keyboard        e   pressed
2018-01-01 05:00:00  keyboard        e  released
2018-01-01 07:00:00  keyboard        e   pressed
2018-01-01 08:00:00  keyboard        e   pressed
2018-01-01 09:00:00  keyboard        e  released
f
                        perif location     event
timestamp                                       
2018-01-01 01:00:00  keyboard        f   pressed
2018-01-01 06:00:00  keyboard        f  released
                        perif location     event  off_time
timestamp                                                 
2018-01-01 00:00:00  keyboard        e   pressed      True
2018-01-01 01:00:00  keyboard        f   pressed     False
2018-01-01 06:00:00  keyboard        f  released      True
2018-01-01 08:0

In [49]:
print(dat_obj.get_move_px())

                     perif location   event  x  y  xdiffsq  ydiffsq  timediff  \
timestamp                                                                       
2018-01-01 00:00:01  mouse      pos  (0, 0)  0  0      NaN      NaN       NaN   
2018-01-01 01:00:01  mouse      pos  (1, 0)  1  0      1.0      0.0    3600.0   
2018-01-01 02:00:01  mouse      pos  (3, 1)  3  1      4.0      1.0    3600.0   
2018-01-01 03:00:01  mouse      pos  (1, 2)  1  2      4.0      1.0    3600.0   
2018-01-01 04:00:01  mouse      pos  (0, 0)  0  0      1.0      4.0    3600.0   
2018-01-01 05:00:01  mouse      pos  (1, 2)  1  2      1.0      4.0    3600.0   

                     distance  velocity  
timestamp                                
2018-01-01 00:00:01       NaN       NaN  
2018-01-01 01:00:01  1.000000  0.000278  
2018-01-01 02:00:01  2.236068  0.000621  
2018-01-01 03:00:01  2.236068  0.000621  
2018-01-01 04:00:01  2.236068  0.000621  
2018-01-01 05:00:01  2.236068  0.000621  


In [51]:
print("Keyboard data:\nraw:\n")
print(dat_obj.keyboard_data)

print(f'Anzahl eingegebener Buchstaben {dat_obj.nr_key_strokes}')
dat_obj.output_string()




Keyboard data:
raw:

                        perif location     event
timestamp                                       
2018-01-01 00:00:00  keyboard        e   pressed
2018-01-01 01:00:00  keyboard        f   pressed
2018-01-01 02:00:00  keyboard        e  released
2018-01-01 03:00:00  keyboard        e   pressed
2018-01-01 04:00:00  keyboard        e   pressed
2018-01-01 05:00:00  keyboard        e  released
2018-01-01 06:00:00  keyboard        f  released
2018-01-01 07:00:00  keyboard        e   pressed
2018-01-01 08:00:00  keyboard        e   pressed
2018-01-01 09:00:00  keyboard        e  released
Anzahl eingegebener Buchstaben 6
TIME:
Timeframe: 30
Endtime: 2018-01-01 09:00:01

Keyboard:
N.o. Activations: 6
Avg. Presstime: 0 days 02:45:00
Avg. Deadtime: 0 days 01:30:00
Mouse:
Avg. Speed: 0.0005524595505555088
